# Kickstarter Startup Analysis

1. How should we design our Kickstarter campaign in order to make our startup funding phase as succesful as possible?
    - **Categorie** (Assumption Tech) - Succesful campaigns
        -> ratio succesful/unsuccesful/live definieren
        -> one-sample 1 tegenover het totaal
        -> two sample 1 tegenover de rest, of Tech vs Design
    - **US vs NL** (Assumption: Silicon Valley vs. the Netherlands)
        -> Average pledged amount in NL hoger OF lager dan in US (1 of 2 sample T test)
    - **Number of backers** (Assumption: popularity of campaign defines popularity of startup)
        -> Ga je voor weinig backers met veel of veel backers met weinig -> verschil in succesful (Hoeveel backers per in je goal)
    -> Zoja/Zonee -> define confidence intervals voor backers voor ons "start-up" plan
    
    - **Pledge amount asked** - Pledged (goal - converted)
    
2. Which pledge amount will lead to the highest possible funding?

3. How could the location of the kickstarter campaign, or the sub-category, influence the success rate of our kickstarter campaign?

**To do**  
~~Elske: Fix git issues  ~~  
Rianne: #Drop duplicates (per project_id, check categorieen - 1, 2 en 3 -- zijn hetzelfde, dus sort_values by pledged amount, keepfirst)  
Rianne: #Currency -> use usd_pledged -> convert_amount can be dropped, fx_rate also, currency also   
Rianne: #Take out live  
~~Elske:  #Canceled and unsuccesful -> Failed~~  
~~Elske:  #Unfold Dict "location" unfold: 'name', 'expanded_country'~~  
~~Elske:  #Create column with average pledge per backer  ~~  
Both: Check for missing values and how to solve them + add code for hypothesis testing

In [1]:
#Importing libraries
import pandas as pd
import json
from datetime import datetime

Download the file <a href="https://drive.google.com/drive/folders/1hIBnfoXrXNW2vF_ka8R8Y0mltCmBjqPZ" target="_blank">here</a>.

In [2]:
#Importing the data using a relative path
df = pd.read_csv('../Kickstarter_analysis/sep2020.csv', converters={'category': eval})
df.head()

,backers_count,blurb,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,id,...,pledged,spotlight,state,state_changed_at,usd_pledged,category_id,category_name,category_parent_id,category_parent_name,project_id
0,61,Support great art! Join us as we re-stage the ...,4618,US,1579292017,USD,1583025192,1.0,3000.0,1568973024,...,4618.0,True,successful,1583025192,4618.0,254,Performances,6.0,Dance,3914215
1,52,JinBucha is a new kind of Brewery in North Par...,3461,US,1446051515,USD,1450118057,1.0,20000.0,1782378629,...,3461.0,False,failed,1450118057,3461.0,307,Drinks,10.0,Food,2198010
2,0,"""Taste The Scent of Tea"" Rose and Blanc Tea Ro...",0,US,1518159717,USD,1520233260,1.0,15000.0,2076142479,...,0.0,False,failed,1520233260,0.0,307,Drinks,10.0,Food,3304826
3,2,Bringing Philly cheesesteaks (and other delici...,25,US,1454217596,USD,1459885844,1.0,2000.0,1416797221,...,25.0,False,failed,1459885844,25.0,311,Food Trucks,10.0,Food,2349147
4,150,a frog plushie keychain and frog butt pin for ...,3275,US,1571342290,USD,1574445600,1.0,800.0,1708708142,...,3275.0,True,successful,1574445600,3275.0,28,Product Design,7.0,Design,3834865


**Step 1:** Data Cleaning and Wrangling
- Transforming unix time stamps to datetime

In [3]:
# Dates transformed [['created_at'], ['deadline'], ['launched_at'], ['state_changed_at']]
df[['created_at','deadline', 'launched_at', 'state_changed_at']].apply(pd.to_datetime, unit='s')

,created_at,deadline,launched_at,state_changed_at
0,2020-01-17 20:13:37,2020-03-01 01:13:12,2020-01-31 01:13:12,2020-03-01 01:13:12
1,2015-10-28 16:58:35,2015-12-14 18:34:17,2015-11-14 18:34:17,2015-12-14 18:34:17
2,2018-02-09 07:01:57,2018-03-05 07:01:00,2018-02-09 20:41:27,2018-03-05 07:01:00
3,2016-01-31 05:19:56,2016-04-05 19:50:44,2016-02-05 20:50:44,2016-04-05 19:50:44
4,2019-10-17 19:58:10,2019-11-22 18:00:00,2019-11-08 18:00:00,2019-11-22 18:00:00
...,...,...,...,...
219683,2019-02-21 18:33:41,2019-03-29 21:00:00,2019-02-27 15:12:10,2019-03-29 21:00:02
219684,2013-03-05 20:13:03,2013-08-10 07:45:40,2013-07-05 07:45:40,2013-08-10 07:45:40
219685,2016-06-21 00:35:01,2016-08-13 21:49:58,2016-07-14 21:49:58,2016-08-13 21:49:58
219686,2015-11-28 19:14:15,2016-01-09 19:19:33,2015-12-10 19:19:33,2016-01-09 19:19:34


- Remove live kickstarter campaings.

- Drop duplicated projects

- Classify cancelled and unsuccesful as both "unsuccesful"  
Since we are looking to launch a succesful campaign for our start-up, 
we categorize the data in two categories: "Succesful" and "Failed" (compromising canceled and failed campaings).

In [9]:
#Value_counts shows there are both canceled and failed campaigns.
df['state'].value_counts()

successful    128156
failed         76137
canceled        9138
live            6257
Name: state, dtype: int64

In [12]:
#Replace canceled campaigns state as 'failed'
df['state'].replace('canceled', 'failed').value_counts()

successful    128156
failed         85275
live            6257
Name: state, dtype: int64

- Create variable 'average_pledge' per backer

In [6]:
df['average_pledge'] = (df['usd_pledged']/df['backers_count'])

- Check for missing values

In [18]:
df.isna().sum()

backers_count                   0
blurb                           8
converted_pledged_amount        0
country                         0
created_at                      0
currency                        0
deadline                        0
fx_rate                         0
goal                            0
id                              0
launched_at                     0
location                      215
name                            0
pledged                         0
spotlight                       0
state                           0
state_changed_at                0
usd_pledged                     0
category_id                     0
category_name                   0
category_parent_id           8300
category_parent_name         8300
project_id                      0
average_pledge              16133
dtype: int64

# Data Exploration

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
#Grouping the df by main category, sorted by sum of converted_pledged amount.
top_pledged = df.loc[:,df['state'] == 'succesful'].groupby(['category_parent_name']).sum().reset_index().sort_values(by='converted_pledged_amount', ascending=False)
top_backers = df.groupby(['category_parent_name']).sum().reset_index().sort_values(by='backers_count', ascending=False)
#Selecting top five from top_pledged and top_backers
top_five_pledged = top_pledged.head(5)
top_five_backers = top_backers.head(5)
pal_pled = sns.color_palette("Greens_d", len(top_five_pledged))
pal_back = sns.color_palette("Reds_d", len(top_five_backers))
rank_pled = top_five_pledged["converted_pledged_amount"].argsort().argsort()
rank_back = top_five_backers["backers_count"].argsort().argsort()
#Plotting the top five
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 4))
sns.barplot(ax =axes[0], x='converted_pledged_amount',y='category_parent_name', data=top_five_pledged, linewidth=0.7, palette=np.array(pal_pled[::+1])[rank_pled])
axes[0].set_title('Converted Amount Pledged')
sns.barplot(ax=axes[1], x='backers_count',y='category_parent_name', data=top_five_backers, linewidth=0.7, palette=np.array(pal_back[::+1])[rank_back])
axes[1].set_title('Number of Backers');

In [ ]:
plt.figure(figsize=(15, 10))
sns.boxplot(x='category_parent_name', y='converted_pledged_amount', hue='state', data=df, orient='v', showfliers=False);
plt.title('Converted Pledged Amount per Category Parent', fontsize=20);
plt.xlabel('Category Parent', fontsize=18)
plt.ylabel('Converted Pledged Amount)', fontsize=18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=15);

#Can we use "proportion of the population" - om significant verschil tussen succesful (tech en de rest)


In [ ]:
df.groupby(['category_name']).sum().sort_values(by='converted_pledged_amount', ascending=False)

In [ ]:
#corr
corr = df['converted_pledged_amount'].corr(df['backers_count'])
#set figure size
plt.figure(figsize=(15, 10))
#plot the data
sns.regplot(data=df,x=df['converted_pledged_amount'],y=df['backers_count'], ci=None)
#format the plot
text = 'r=%s' % (corr)
plt.text(1, 19, text, fontsize=14)
plt.title('', fontsize=20);
plt.xlabel('', fontsize=18)
plt.ylabel('', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18);

# Statistical Analysis

In [25]:
#Confidence Intervals:
# To define between pledge amount x and x we should ask
# To define between number of backers x and x we should aim for

# One-sample
# 1 category opposed to population mean

# Two-sample
#2 categories (tech vs. food) and Netherlands/Europe vs. US